# Custom EnergyPlus Evaluator functions
This notebook shows how to write a custom function that gets applied to the EnergyPlus outputs.

In [ ]:

from besos import eppy_funcs as ef
from besos.evaluator import EvaluatorEP
from besos.objectives import MeterReader, time_series_values
from besos.parameters import FieldSelector, Parameter, RangeParameter
from besos.problem import EPProblem

### Set up an EnergyPlus Evaluator
Here is a standard EnergyPlus Evaluator for editing the lighting power density for the default building and getting the electricity demand time series.
By default, we get the sum of the variable we specify.

In [ ]:
building = ef.get_building()
EPparameters = [
    Parameter(
        FieldSelector("Lights", "*", "Watts per Zone Floor Area"),
        value_descriptor=RangeParameter(8, 12),
        name="Lights Watts/Area",
    )
]
EPobjectives = MeterReader("Electricity:Facility")
problem = EPProblem(EPparameters, EPobjectives)
evaluator = EvaluatorEP(problem, building)
result = evaluator([8])
result

### Time series data

Sometimes we want to get back an entire time series, for example for linking to the Energy Hub model (see [here](../EnergyHub/EPtoEHEvaluators.ipynb)).
For that we can specify `func=time_series_values`, which comes as part of besos.objectives.

In [ ]:
EPobjectives = MeterReader("Electricity:Facility", func=time_series_values)
problem = EPProblem(EPparameters, EPobjectives)
evaluator = EvaluatorEP(problem, building)
result = evaluator([8])
result

### Define a custom function
Now we will define the function that we will apply to the EnergyPlus output data.
The function `highloadhoursfunc` counts the hours in which the load is above a threshold.

In [ ]:
def highloadhoursfunc(result):
    threshold = 1e7
    highloadhours = sum(result.data["Value"] >= threshold)
    return highloadhours

By specifying `func=highloadhoursfunc`, the result is the number of hours in which `Electricity:Facility` is above the threshold.

In [ ]:
EPobjectives = MeterReader("Electricity:Facility", func=highloadhoursfunc)
problem = EPProblem(EPparameters, EPobjectives)
evaluator = EvaluatorEP(problem, building)
result = evaluator([8])
result